# Person.ipynb

Demonstration notebook for Text Extensions for Pandas.

This notebook evaluates the effectiveness of a person name extractor using the facilities of Text Extensions for Pandas.

Instructions to run:
1. (optional) Use the script `env.sh` at the root of this project to create an Anaconda environment `pd` with required packages. Activate this environment by typing `conda activate pd`.
1. From a shell window at the root of the project, start up JupyterLab by typing `jupyter lab`
1. Inside JupyterLab, navigate to the `notebooks` directory and open up this notebook. You should now be able to run the code in this notebook.

In [1]:
# INITIALIZATION BOILERPLATE

# The Jupyter kernel for this notebook usually starts up inside the notebooks
# directory, but the text_extensions_for_pandas package code is in the parent
# directory. Add that parent directory to the front of the Python include path.
import sys
if (sys.path[0] != ".."):
    sys.path[0] = ".."
    
# Libraries
import numpy as np
import pandas as pd
import regex
import spacy
spacy_language_model = spacy.load("en_core_web_sm")
import textwrap

# And of course we need the text_extensions_for_pandas library itself.
import text_extensions_for_pandas as tp

In [2]:
# Example document text courtesy https://en.wikipedia.org/wiki/Monty_Python_and_the_Holy_Grail
# License: CC-BY-SA
with open ("../resources/holy_grail.txt", "r") as f:
    doc_text = f.read()
    pass
 
# Parse the document text with SpaCy, then convert the results to a dataframe
token_features = tp.make_tokens_and_features(doc_text, spacy_language_model)
token_features

,id,char_span,token_span,lemma,pos,tag,dep,head,shape,ent_iob,ent_type,is_alpha,is_stop,sentence
0,0,"[0, 5): 'Monty'","[0, 1): 'Monty'",Monty,PROPN,NNP,compound,1,Xxxxx,B,PERSON,True,False,"[0, 52): 'Monty Python and the Holy Grail is a..."
1,1,"[6, 12): 'Python'","[1, 2): 'Python'",Python,PROPN,NNP,nsubj,6,Xxxxx,I,PERSON,True,False,"[0, 52): 'Monty Python and the Holy Grail is a..."
2,2,"[13, 16): 'and'","[2, 3): 'and'",and,CCONJ,CC,cc,1,xxx,O,,True,True,"[0, 52): 'Monty Python and the Holy Grail is a..."
3,3,"[17, 20): 'the'","[3, 4): 'the'",the,DET,DT,det,5,xxx,B,ORG,True,True,"[0, 52): 'Monty Python and the Holy Grail is a..."
4,4,"[21, 25): 'Holy'","[4, 5): 'Holy'",Holy,PROPN,NNP,compound,5,Xxxx,I,ORG,True,False,"[0, 52): 'Monty Python and the Holy Grail is a..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1078,1078,"[5306, 5314): 'officers'","[1078, 1079): 'officers'",officer,NOUN,NNS,pobj,1076,xxxx,O,,True,False,"[1071, 1083): 'The movie ends with one of the ..."
1079,1079,"[5315, 5323): 'breaking'","[1079, 1080): 'breaking'",break,VERB,VBG,acl,1078,xxxx,O,,True,False,"[1071, 1083): 'The movie ends with one of the ..."
1080,1080,"[5324, 5327): 'the'","[1080, 1081): 'the'",the,DET,DT,det,1081,xxx,O,,True,True,"[1071, 1083): 'The movie ends with one of the ..."
1081,1081,"[5328, 5334): 'camera'","[1081, 1082): 'camera'",camera,NOUN,NN,dobj,1079,xxxx,O,,True,False,"[1071, 1083): 'The movie ends with one of the ..."


In [4]:
from text_extensions_for_pandas.array.tensor import TensorArray

x = np.random.rand(10, 3)
x

a = TensorArray(x)
a

TypeError: object of type 'numpy.float64' has no len()

AttributeError: 'TensorArray' object has no attribute 'as_frame'

In [3]:
# We can extract out all unique sentence spans by aggregating the "sentence" 
# column of the above dataframe:
sentences = pd.DataFrame({"sentence": token_features["sentence"].unique()})
sentences.head(10)

,sentence
0,"[0, 52): 'Monty Python and the Holy Grail is a..."
1,"[52, 76): 'It was conceived during the hiatus ..."
2,"[76, 122): 'While the group's first film, And ..."
3,"[122, 139): 'Thirty years later, Idle used the..."
4,"[139, 159): 'Monty Python and the Holy Grail g..."
5,"[159, 190): 'In the US, it was selected as the..."
6,"[190, 203): 'In the UK, readers of Total Film ..."
7,"[203, 227): 'the fifth-greatest comedy film of..."
8,"[227, 254): 'In AD 932, King Arthur and his sq..."
9,"[254, 313): 'Along the way, he recruits Sir Be..."


In [4]:
# The "ent_iob" and "ent_type" fields contain entity tags in 
# Inside-Outside-Beginning (IOB) format.
# Text Extensions for Pandas has a built-in function to convert 
# IOB tagged data to spans of entities.
entities = tp.iob_to_spans(token_features)
entities

,token_span,ent_type
0,"[0, 2): 'Monty Python'",PERSON
1,"[3, 6): 'the Holy Grail'",ORG
2,"[8, 9): '1975'",DATE
3,"[9, 10): 'British'",NORP
4,"[22, 24): 'Monty Python'",ORG
...,...,...
110,"[989, 990): 'Bedevere'",ORG
111,"[992, 996): 'the Castle of Aarrgh'",ORG
112,"[1005, 1006): 'French'",NORP
113,"[1054, 1055): 'Arthur'",PERSON


In [5]:
# Let's look at just the entities tagged "PERSON"
person_entities = entities[entities["ent_type"] == "PERSON"]
person_entities.head()

,token_span,ent_type
0,"[0, 2): 'Monty Python'",PERSON
5,"[27, 29): 'Graham Chapman'",PERSON
6,"[30, 32): 'John Cleese'",PERSON
7,"[33, 35): 'Terry Gilliam'",PERSON
8,"[36, 38): 'Eric Idle'",PERSON


In [6]:
# Use the TokenSpanArray's built-in HTML rendering to look at these
# PERSON entities in the context of the document.
person_entities["token_span"].values

,begin,end,begin_token,end_token,covered_text
0,0,12,0,2,Monty Python
1,153,167,27,29,Graham Chapman
2,169,180,30,32,John Cleese
3,182,195,33,35,Terry Gilliam
4,197,206,36,38,Eric Idle
5,208,219,39,41,Terry Jones
6,224,237,42,44,Michael Palin
7,267,272,50,51,Jones
8,376,390,69,72,Monty Python's
9,722,734,139,141,Monty Python


In [7]:
# Load gold standard labels in IOB format from a CSV file
person_gold_iob = pd.read_csv("../resources/holy_grail_person.csv")

# Pull in token offsets from our token_features dataframe
person_gold_iob["token_span"] = token_features["token_span"].values
person_gold_iob["char_span"] = token_features["char_span"].values
person_gold_iob.iloc[25:35]

,token_text,ent_iob,token_span,char_span
0,Monty,O,"[0, 1): 'Monty'","[0, 5): 'Monty'"
1,Python,O,"[1, 2): 'Python'","[6, 12): 'Python'"
2,and,O,"[2, 3): 'and'","[13, 16): 'and'"
3,the,O,"[3, 4): 'the'","[17, 20): 'the'"
4,Holy,O,"[4, 5): 'Holy'","[21, 25): 'Holy'"


In [8]:
# Convert from IOB format to spans of entities
person_gold = tp.iob_to_spans(person_gold_iob, entity_type_col_name=None)
person_gold.head()

,token_span
0,"[27, 29): 'Graham Chapman'"
1,"[30, 32): 'John Cleese'"
2,"[33, 35): 'Terry Gilliam'"
3,"[36, 38): 'Eric Idle'"
4,"[39, 41): 'Terry Jones'"


In [9]:
# Find all the spans that are in both the extractor's answer set and the gold standard
person_intersection = person_gold.merge(person_entities)
person_intersection.head()

,id,char_span,token_span,lemma,pos,tag,dep,head,shape,ent_iob,ent_type,is_alpha,is_stop,sentence
254,254,"[1257, 1262): 'Along'","[254, 255): 'Along'",along,ADP,IN,prep,259,Xxxxx,O,,True,True,"[254, 313): 'Along the way, he recruits Sir Be..."
255,255,"[1263, 1266): 'the'","[255, 256): 'the'",the,DET,DT,det,256,xxx,O,,True,True,"[254, 313): 'Along the way, he recruits Sir Be..."
256,256,"[1267, 1270): 'way'","[256, 257): 'way'",way,NOUN,NN,pobj,254,xxx,O,,True,False,"[254, 313): 'Along the way, he recruits Sir Be..."
257,257,"[1270, 1271): ','","[257, 258): ','",",",PUNCT,",",punct,259,",",O,,False,False,"[254, 313): 'Along the way, he recruits Sir Be..."
258,258,"[1272, 1274): 'he'","[258, 259): 'he'",-PRON-,PRON,PRP,nsubj,259,xx,O,,True,True,"[254, 313): 'Along the way, he recruits Sir Be..."
259,259,"[1275, 1283): 'recruits'","[259, 260): 'recruits'",recruit,VERB,VBZ,ccomp,271,xxxx,O,,True,False,"[254, 313): 'Along the way, he recruits Sir Be..."
260,260,"[1284, 1287): 'Sir'","[260, 261): 'Sir'",Sir,PROPN,NNP,compound,261,Xxx,O,,True,False,"[254, 313): 'Along the way, he recruits Sir Be..."
261,261,"[1288, 1296): 'Bedevere'","[261, 262): 'Bedevere'",Bedevere,PROPN,NNP,dobj,259,Xxxxx,B,PERSON,True,False,"[254, 313): 'Along the way, he recruits Sir Be..."
262,262,"[1297, 1300): 'the'","[262, 263): 'the'",the,DET,DT,det,263,xxx,I,PERSON,True,True,"[254, 313): 'Along the way, he recruits Sir Be..."
263,263,"[1301, 1305): 'Wise'","[263, 264): 'Wise'",Wise,PROPN,NNP,appos,261,Xxxx,I,PERSON,True,False,"[254, 313): 'Along the way, he recruits Sir Be..."


In [10]:
# Let's compute precision and recall, just on this document.
# Of course, in a real use case, we would be computing these values on a 
# development holdout set of documents while tuning the model, then
# computing them again on a validation set during final testing.
# We use a single document here to show that it is straightforward 
# to collect the necessary information using Pandas.
num_true_positives = len(person_intersection.index)
num_entities = len(person_gold.index)
num_entities_extracted = len(person_entities.index)

precision = num_true_positives / num_entities_extracted
recall = num_true_positives / num_entities
F1 = 2.0 * (precision * recall) / (precision + recall)

print(
"""Number of correct answers: {}
Number of entities identified: {}
Actual number of entities: {}
Precision: {:1.2f}
Recall: {:1.2f}
F1: {:1.2f}""".format(num_true_positives, num_entities, num_entities_extracted, precision, recall, F1))

In [ ]:
# That seems a bit low. Let's look at the false positives.
false_positives = person_entities[~person_entities["token_span"].isin(person_gold["token_span"])]
false_positives

In [ ]:
# Hmm, aside from the first three, most of these appear to be partial matches.
# Let's recompute precision and recall giving credit for partial matches.
# We start by finding out how many spans in person_entities["token_span"]
# are contained within a span from person_gold["token_span"]
looser_intersection = tp.contain_join(person_gold["token_span"], person_entities["token_span"],
                                      "gold", "extracted")
looser_intersection

In [ ]:
# Note that there are some duplicates (rows 23 and 24, for example).
# Use the number of unique values in the "gold" column to compute
# how many partial or complete matches of an entity we found.
num_unique_matches = len(looser_intersection["gold"].unique())
num_unique_matches

In [ ]:
# Recompute precision, recall, and F1 score on this looser basis.
# Again, in a real use case we would be doing this operation on a holdout set of
# multiple documents. The point here is that the core computations all map
# easily into Pandas.
num_true_positives = num_unique_matches
num_entities = len(person_gold.index)
num_entities_extracted = len(person_entities.index)

precision = num_true_positives / num_entities_extracted
recall = num_true_positives / num_entities
F1 = 2.0 * (precision * recall) / (precision + recall)

print(
"""Number of correct answers: {}
Number of entities identified: {}
Actual number of entities: {}
Precision: {:1.2f}
Recall: {:1.2f}
F1: {:1.2f}""".format(num_true_positives, num_entities, num_entities_extracted, precision, recall, F1))

In [ ]:
# Let's drill down on those partial matches to see what's causing them
# (at least on this one document)
partial_matches = looser_intersection[looser_intersection["gold"].values != looser_intersection["extracted"].values].reset_index(drop=True)
partial_matches

In [ ]:
# Hmm, there seems to be some clustering of the matches. Let's see how
# they map onto the sentences of the document.
extracted_sentence = tp.contain_join(sentences["sentence"], partial_matches["extracted"],
                                     first_name="sentence")
partial_matches["sentence"] = extracted_sentence["sentence"].values
partial_matches

In [ ]:
# Looks like 1/3 of the partial matches on this document are clustered in a 
# single problem sentence. Let's take a closer look at that sentence.
sentence_span = partial_matches["sentence"].loc[0]
sentence = token_features[token_features["sentence"] == sentence_span]
sentence.head(10)

In [ ]:
# Use SpaCy to render the dependency parse of the sentence
tp.render_parse_tree(sentence)

In [ ]:
# That's a lot of parse tree! Let's cut that down to the tokens
# that cover entities from the gold standard data.
entity_tokens = tp.contain_join(person_gold["token_span"], sentence["token_span"],
                                "entity", "token_span")
entity_tokens.head(10)

In [ ]:
# Extract out and display the part of the dependency parse that covers just those tokens
mask = token_features["token_span"].isin(entity_tokens["token_span"])
partial_parse = token_features[mask]
tp.render_parse_tree(partial_parse)

In [ ]:
# With the filtered parse tree, two things pop out:
# 1. The dependency parser model finds information about proper noun phrases
#    that the NER model does not catch.
# 2. The phrase "Sir Not-Appearing-in-this-Film" causes the dependency parser 
#    model to go off the rails.
#
# Let's see if we can combine the results of the two models to get more accurate
# spans.
# First, let's use some Gremlin to extract out the compound proper nouns from
# the parse tree. We'll do this at the document level.
g = tp.token_features_to_traversal(token_features)
compound_proper_nouns = (
    g.V()  # Start with all vertices.
    .has("tag", "NNP")  # Filter out those not tagged NNP (proper noun).
    .has("dep", "compound").as_("src")  # Filter out those without a dependency link of type "compound".
    .out()  # Follow the outgoing link to the parent node.
    .has("tag", "NNP").as_("dest")  # Filter paths where the parent node is not a proper noun.
    .select("src", "dest").by("token_span")  # Return parents of tokens
).toDataFrame()
# Add a third column with the combined span
compound_proper_nouns["phrase"] = compound_proper_nouns["src"] + compound_proper_nouns["dest"]
compound_proper_nouns.head(10)

In [ ]:
# Let's find the cases where a compound proper noun from the deep parser
# overlaps (but does not exactly match) with a person entity from the 
# named entity recognizer.
overlap = tp.overlap_join(compound_proper_nouns["phrase"], person_entities["token_span"],
                          first_name="compound_phrase", second_name="person")
strict_overlap = overlap[~overlap["compound_phrase"].isin(person_entities["token_span"])].reset_index(drop=True)
strict_overlap

In [ ]:
# Use these pairs of spans to build up expanded person spans
strict_overlap["expanded_person"] = strict_overlap["compound_phrase"] + strict_overlap["person"]
strict_overlap

In [ ]:
# If we just added these expanded spans back to our original set of 
# entities, we would get overlapping results. Find and filter out the 
# results from the original entities that overlap with our expanded
# person entities.
to_filter = tp.overlap_join(strict_overlap["expanded_person"], person_entities["token_span"],
                            first_name="expanded_person", second_name="token_span")
to_filter

In [ ]:
# Remove the contents of to_filter and add the contents of strict_overlap to
# our original set of persons
filtered = person_entities["token_span"][~person_entities["token_span"].isin(to_filter["token_span"])]
person_entities_2 = pd.DataFrame({"token_span": 
                                  pd.concat([filtered, strict_overlap["expanded_person"]])
                                    .sort_values()
                                    .reset_index(drop=True)})
person_entities_2

In [ ]:
# Let's see what this correction does to the exact-match precision and recall
person_intersection_2 = person_gold.merge(person_entities_2)
num_true_positives = len(person_intersection_2.index)
num_entities = len(person_gold.index)
num_entities_extracted = len(person_entities_2.index)

precision = num_true_positives / num_entities_extracted
recall = num_true_positives / num_entities
F1 = 2.0 * (precision * recall) / (precision + recall)

print(
"""Number of correct answers: {}
Number of entities identified: {}
Actual number of entities: {}
Precision: {:1.2f}
Recall: {:1.2f}
F1: {:1.2f}""".format(num_true_positives, num_entities, num_entities_extracted, precision, recall, F1))

Here we've just shown that you can quickly combine the results of multiple
models using Pandas and Gremlin.

It's important to note that the improvement in precision may or
may not generalize to the other documents of the corpus. In a real use case, we would need to 
validate this approach against a development set of test documents. If this
simple hybrid approach works well there, an appropriate next step would be 
to retrain the NER model using the dependency parser's "compound" tags as 
an additional feature.

In [ ]:
# Now precision is looking pretty good, but recall is kind of low.
# Let's examine the missing results.
missing_results_mask = ~(person_gold["token_span"].isin(looser_intersection["gold"]))
missing_results = person_gold[missing_results_mask]
missing_results